In [12]:
from tkinter import *
from tkinter import ttk
import glob
import os
import statistics as st
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import time
import string
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report, accuracy_score
from sklearn.utils import shuffle
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [13]:
class User:
    def __init__ (self):
           self.name = ''
           self.algName = ''
           self.comment = ''
           self.prediction = ''
           self.product = ''
    
    def getProduct(self):
        return self.product
    
    def setProduct(self,product):
        self.product = product
        
    def getPrediction(self):
        return self.prediction
    
    def setPrediction(self,prediction):
        self.prediction = prediction
        
    def getComment(self):
        return comment
    def setComment(self,comment):
        self.comment = comment
        
    def getAlgName(self):
        return self.algName
    
    def setAlgName(self,algName):
        self.algName = algName
        
    def getName(self):
        return self.name
    
    def setName(self,name):
        self.name = name

In [14]:
def implementAlg():
    data = User()
    fjalia_prej_inputit = data.getComment()
    fjalia_e_procesuar = docs_preprocessor(fjalia_prej_inputit)

    instance = tokenizer.texts_to_sequences(fjalia_e_procesuar)

    flat_list = []
    for sublist in instance:
        for item in sublist:
            flat_list.append(item)

    flat_list = [flat_list]

    instance = pad_sequences(flat_list, maxlen=15)

    clf_DT.predict(instance)
    def prediction_string(text, alg):
        if alg == 1:
            y_pred = clf_NB.predict(text)
            return y_pred

        elif alg == 2:
            y_pred = clf_RF.predict(text)
            return y_pred

        elif alg == 3:
            y_pred = clf_Ada.predict(text)
            return y_pred

        elif alg == 4:
            y_pred = clf_DT.predict(text)
            return y_pred
    data.setPrediction = prediction_string(instance, data.getAlgName())

In [15]:
data = pd.read_csv("Dataseti\GrammarandProductReviews.csv")

In [16]:
def revPurchased():
    revPurchTrue = data[data['reviews.didPurchase'] == True]['reviews.didPurchase'].reset_index().drop('index',axis=1).count()
    revPurchFalse = data[data['reviews.didPurchase'] == False]['reviews.didPurchase'].reset_index().drop('index',axis=1).count()
    dataToPerform = pd.DataFrame({"RevPurchasedTrue":revPurchTrue,"RevPurchasedFalse":revPurchFalse}).T.rename({'reviews.didPurchase':'Total'},axis=1)
    
    figure , ax= plt.subplots(figsize=(10,6))
    ax = sns.barplot(x=dataToPerform.index, y='Total',data=dataToPerform, hue='Total',palette="rocket",dodge=False)
    ax.set_title("Review Purchased/Not_Purchased", fontsize=13)
    
    return figure

In [17]:
def revRecommanded():
    revRecomanded = data[data['reviews.doRecommend'] == True]['reviews.doRecommend'].reset_index().drop('index',axis=1).count()
    revNotRecomanded = data[data['reviews.doRecommend'] == False]['reviews.doRecommend'].reset_index().drop('index',axis=1).count()
    dataToPerform = pd.DataFrame({"RevRecomanded":revRecomanded,"RevNotRecomanded":revNotRecomanded}).T.rename({'reviews.doRecommend':'Total'},axis=1)
    
    figure,ax= plt.subplots(figsize=(10,6))
    plt.pie(x=dataToPerform['Total'], autopct="%.1f%%", explode=(0, 0.17), labels=['RevRecomanded','RevNotRecomanded'], pctdistance=0.5,
            colors=['#b8b5ff','#f25287'])
    plt.title("Review Recomanded/Not_Recomanded", fontsize=13);
    
    return figure

In [18]:
def totalRatings():
    reviewsRating = data['reviews.rating'].value_counts().sort_index().rename(index='Ratings').to_frame()
    sns.set_style("whitegrid")
    figure,ax = plt.subplots(figsize=(10,6))
    ax = sns.barplot(x="Ratings", y=reviewsRating.index, data=reviewsRating, ci=None, palette="rocket",orient='h' )
    ax.set_title("Total Ratings", fontsize=15)
    ax.set_xlabel ("Ratings")
    ax.set_ylabel ("Ratings Type")
    for rect in ax.patches:
        ax.text(rect.get_width(), rect.get_y() + rect.get_height() / 2,int(rect.get_width()), weight='bold')
    
    return figure

In [19]:
def totalSalesPerBrand():
    brand = data.loc[data['reviews.didPurchase'] == True,['brand','reviews.didPurchase']].reset_index().drop('index',axis=1)
    brandSales = brand.groupby('brand').aggregate({'reviews.didPurchase' : 'count'}).reset_index().rename(columns = {'brand' : 'Brand' , 'reviews.didPurchase' : 'Total Sales For Brand'})
    mainBrands = brandSales[brandSales['Total Sales For Brand']>10]
    otherBrands = brandSales[brandSales['Total Sales For Brand']<10].aggregate({'Total Sales For Brand':'sum'}).rename('Others')
    dataToPerform = mainBrands.set_index('Brand').append(otherBrands, ignore_index=False).reset_index()
    
    sns.set_style('dark')
    figure,ax = plt.subplots(figsize =(10,6))
    ax = sns.relplot(x='Brand',y='Total Sales For Brand',data=dataToPerform,kind='line',aspect = 3,color = '#126e82') 
    ax.set(xticks=dataToPerform['Brand'])
    ax.set_xticklabels(dataToPerform['Brand'],rotation=90,fontsize=12)
    
    return figure

In [20]:
def totalSalesPerYear():
    years = pd.to_datetime(data['dateAdded']).dt.year
    sales = pd.Series(data['reviews.didPurchase'])

    dataToPerform =pd.DataFrame({'Year':years,"Total Sales Per Year":sales}).groupby('Year').aggregate({'Total Sales Per Year':'count'})
    sns.set_style("darkgrid")
    figure,ax = plt.subplots(figsize=(10,6))
    ax = sns.barplot(x=dataToPerform.index, y="Total Sales Per Year", data=dataToPerform, ci=None, palette="rocket",orient='v',
                         hue = 'Total Sales Per Year',dodge=False )
    ax.set_title("Sales Per Year", fontsize=15)
    ax.set_xlabel ("Year")
    ax.set_ylabel ("Total Sales")
    
    return figure
    

In [21]:
def totalSalesBrandRecommanded():
    brand = data.loc[(data['reviews.didPurchase'] == True) & (data['reviews.doRecommend'] == True) ,
    ['brand','reviews.didPurchase','reviews.doRecommend']].reset_index().drop('index',axis=1)
    brandSales = brand.groupby('brand').aggregate({'reviews.didPurchase' : 'count' }).reset_index().rename(columns = {'brand' : 'Brand' , 'reviews.didPurchase' : 'Total Sold And Recomanded Brand'})
    mainBrands = brandSales[brandSales['Total Sold And Recomanded Brand']>10]
    otherBrands = brandSales[brandSales['Total Sold And Recomanded Brand']<10].aggregate({'Total Sold And Recomanded Brand':'sum'}).rename('Others')
    dataToPerform = mainBrands.set_index('Brand').append(otherBrands, ignore_index=False).reset_index()
    
    figure ,ax = plt.subplots(figsize=(30,9))
#     sns.set_style('dark')
    ax = sns.relplot(x='Brand',y='Total Sold And Recomanded Brand',data=dataToPerform,kind='line',aspect = 3,color = '#126e82') 
    ax.set(xticks=dataToPerform['Brand'])
    ax.set_xticklabels(dataToPerform['Brand'],rotation=90,fontsize=12,color='#132c33')
    plt.suptitle("Total Sold And Recomanded Brands" , y=1.01)

    return figure

In [22]:
def totalSalesPerMonth():
    dates = pd.to_datetime(data['dateAdded']).dt
    sales = pd.Series(data['reviews.didPurchase'])

    dataToPerform = pd.DataFrame({'Year':dates.year, 'Month':dates.month , 'Day':dates.day , 'Sales':sales})

    dataFor2014 = dataToPerform[dataToPerform['Year'] == 2014].groupby('Month').aggregate({'Sales':'count'})
    dataFor2015 = dataToPerform[dataToPerform['Year'] == 2015].groupby('Month').aggregate({'Sales':'count'})
    dataFor2016 = dataToPerform[dataToPerform['Year'] == 2016].groupby('Month').aggregate({'Sales':'count'})
    dataFor2017 = dataToPerform[dataToPerform['Year'] == 2017].groupby('Month').aggregate({'Sales':'count'})

    fig = plt.figure()
    gs = fig.add_gridspec(2, 2, hspace=0, wspace=0)
    (ax1, ax2), (ax3, ax4) = gs.subplots(sharex='col', sharey='row')
    fig.suptitle('Sales For Months Seperated by Years')
    ax1.plot(dataFor2014)
    ax2.plot(dataFor2015, 'tab:orange')
    ax3.plot(dataFor2016, 'tab:green')
    ax4.plot(dataFor2017, 'tab:red')
    
    return fig
    

In [23]:
def totalRecommandations():
    years = pd.to_datetime(data['dateAdded']).dt.year
    sales = pd.Series(data['reviews.doRecommend'])
    dataToPerform =pd.DataFrame({'Year' : years , "Recommendations Per Year":sales}).groupby('Year').aggregate({'Recommendations Per Year':'count'})
    
    figure ,ax= plt.subplots(figsize = (10,6))
    plt.plot(dataToPerform.index, 'Recommendations Per Year' ,data=dataToPerform, marker='o', markerfacecolor='blue', markersize=12, color='skyblue', linewidth=4)
    plt.xticks(range(dataToPerform.index[0],dataToPerform.index[len(dataToPerform.index)-1]+1))
    plt.title("Recommandations Per Year", fontsize=15)
    plt.xlabel('Years')
    plt.ylabel('Recommandations Per Year')
    plt.legend()
    
    return figure


In [24]:
class Root(Tk):
    def __init__(self,i):
        super(Root,self).__init__()
        self.i=i
        self.minsize(640,400)
        self.matplotCanvas()

    def matplotCanvas(self):
        canvas=FigureCanvasTkAgg(self.i,self)
        canvas.draw()
        canvas.get_tk_widget().pack(side = BOTTOM , fill = BOTH, expand = True)

    
def callback():
    pass

def review_purchased():
    i=revPurchased()
    root=Root(i)
    root.mainloop()
    
def review_recommanded():
    i=revRecommanded()
    root=Root(i)
    root.mainloop()
    
def total_ratings():
    i=totalRatings()
    root=Root(i)
    root.mainloop()
    
def total_salesBrand():
    i=totalSalesPerBrand()
    root=Root(i)
    root.mainloop()

def total_salesBrandRecommanded():
    i=totalSalesBrandRecommanded()
    root=Root(i)
    root.mainloop()
    
def sales_perYear():
    i=totalSalesPerYear()
    root=Root(i)
    root.mainloop()

def sales_perMonth():
    i=totalSalesPerMonth()
    root=Root(i)
    root.mainloop()
def recomma_ndations():
    i=totalRecommandations()
    root=Root(i)
    root.mainloop()

In [25]:
def addReview():
    pass
    

In [26]:
def setOnclick():
    data = User()
    if (data.getAlgName == 1):
        data.setProduct = "Product is bought!" if data.getPrediction() == 1 else "Product is not bought!"  
    elif (data.getAlgName == 2):
        data.setProduct = "Product is bought!" if data.getPrediction() == 1 else "Product is not bought!"
    elif (data.getAlgName == 3):
        data.setProduct = "Product is bought!" if data.getPrediction() == 1 else "Product is not bought!"
    elif(data.getAlgName == 4):
        data.setProduct = "Product is bought!" if data.getPrediction() == 1 else "Product is not bought!"
    else:
        data.setProduct = 'You dident enter nothing '
    

In [27]:
def predictFunc():
    data = User()
    
    window = Toplevel()
    
    commentLabel = Label(window,text='Enter Text', bg="#caf7e3", width="300", height="2", font=("Calibri", 13))
    commentLabel.pack(pady=10)
    
    userInput =Entry(window, textvariable= 'userInput')
    userInput.pack(pady=20)
    Label(window,text="").pack()
    
#     userInput =Entry(root, textvariable= 'userInput')
#     userInput.pack()
#     Label(root,text="").pack()
    v = IntVar()
    v.set(1)  # initializing the choice, i.e. Python

    languages = [("Naive Bayes", 1),
             ("Decision Tree", 2),
                 ("Random Forest", 3),
                 ("Ada Boosting", 4)]

    def ShowChoice():
        print(v.get())

    Label(window, 
             text="""Choose your algorithem:""",
             justify = CENTER,
             padx = 20).pack()

    for language, val in languages:
        Radiobutton(window, 
                       text=language,
                       padx = 20, 
                       variable=v, 
                       command=ShowChoice,
                       value=val).pack(anchor=W)
        
    btnClick = Button(window, text="Click", height="1", width="15", command = setOnclick) 
    btnClick.pack(pady=10)
    
    
    predictedLabel = Label(window,text=data.getProduct(), bg="#caf7e3", width="300", height="2", font=("Calibri", 13))
    predictedLabel.pack(pady=10)
    
    data.setComment(userInput.get())
    data.setAlgName(v.get())

    window.geometry("300x450")

In [28]:
#window that holds the button for stats
def statsButtons():
    root = Toplevel()
               
    reviewPurchased = Button(root, text="Review Purchased / Not Purchased", height="1", width="30", command = review_purchased) 
    reviewPurchased.pack(pady=10)
    Label(text='').pack();
    
    reviewRecommanded = Button(root, text="Review Recommanded / Not Recommanded", height="1", width="30", command = review_recommanded) 
    reviewRecommanded.pack(pady=10)
    Label(text='').pack();
    
    totalRatings = Button(root, text="Total Ratings", height="1", width="30", command = total_ratings) 
    totalRatings.pack(pady=10)
    Label(text='').pack();
    
    totalSalesBrand = Button(root, text="Total Sales Per Brand", height="1", width="30", command = total_salesBrand) 
    totalSalesBrand.pack(pady=10)
    Label(text='').pack();
    
    totalSalesBrandRecommanded = Button(root, text="Total Sold And Recommanded", height="1", width="30", command = total_salesBrandRecommanded) 
    totalSalesBrandRecommanded.pack(pady=10)
    Label(text='').pack();
    
    salesPerYear = Button(root, text="Sales Per Year", height="1", width="30", command = sales_perYear) 
    salesPerYear.pack(pady=10)
    Label(text='').pack();
    
    salesPerMonth = Button(root, text="Sales Per Month Seperated by years", height="1", width="30", command = sales_perMonth) 
    salesPerMonth.pack(pady=10)
    Label(text='').pack();
    
    recommandations = Button(root, text="Recomandations Per Year", height="1", width="15", command = recomma_ndations) 
    recommandations.pack(pady=10)
    
    root.geometry("300x450")
    

In [31]:
def dic_imgs():
    imgs = {}
    for i in glob.glob("icons/*.png"):
        pathfile = i
        i = os.path.basename(i)
        name = i.split(".")[0]
        imgs[name] = PhotoImage(file=pathfile)
        if name == "plus":
            imgs[name] = imgs[name].subsample(2)
    return imgs


# create a Form label 
root = Tk()
root.title("Account Login") # set the title of GUI window

obj =User()

label = Label(root,text='Enter your username to continue', bg="#caf7e3", width="300", height="2", font=("Calibri", 13))
label.pack()
Label(root,text="").pack()
 
# create Login Button 
usernameInput =Entry(root, textvariable= 'username')
usernameInput.pack()
Label(root,text="").pack()
 

def openMainWindow():
    obj.setName(usernameInput.get())
    # Toplevel object which will 
    # be treated as a new window
    root.destroy()
    
    
    newWindow = Tk()
    
    imgs = dic_imgs()    
    
    img = PhotoImage(file="icons\Besi2.png")
    backgroundImg = Label(newWindow,image=img)
    backgroundImg.place(x=0, y=0)

    labelUsername = Label(newWindow, text ="Hi "+obj.getName(), font=("Calibri", 15), bg = 'white')
    labelUsername.pack(side=TOP, anchor=NW,pady = 10 , padx= 20)
    
    labelForUser = Label(newWindow, text ="What Can We Do for u today ? " , font=("Calibri", 15), bg = 'white')
    labelForUser.pack()
    
    #####################
    imgs = dic_imgs()
    
    toolbar = Frame(newWindow)
    toolbar.pack(side=TOP, fill=X,pady=30)
 
    b1 = Button(
    toolbar,
    relief=FLAT,
    compound = LEFT,
    text="Stats",
    command=statsButtons,
    image=imgs["statsSign"])
    b1.pack(side=BOTTOM, padx=0, pady=0)

    
    b2 = Button(
    toolbar,
    text="Give a Review",
    compound = LEFT,
    command=addReview,
    relief=FLAT,
    image=imgs["reviewSign"])
    b2.pack(side=BOTTOM, padx=0, pady=0)
    
    b3 = Button(
    toolbar,
    text="Predict",
    compound = LEFT,
    command=predictFunc,
    relief=FLAT,
    image=imgs["dollarSign"])
    b3.pack(side=BOTTOM, padx=0, pady=0)
    
    
   ##########################
    
    btn = Button(newWindow, text="Exit", height="1", width="15", command = callback) 
    btn.pack(pady=10)
    
    
    
    
    
    labelForButtons = Label(newWindow,bg =None)
    
    newWindow.geometry("750x450")
    newWindow.title("New Window")
    newWindow.mainloop()


# create a register button
btn = Button(root, text="Continue", height="1", width="15", command = openMainWindow) 
btn.pack()

root.geometry("300x250")
root.mainloop() # start the GUI
 
# main_account_screen() # call the main_account_screen() function